## Estrutura do Processo:
### **Extração**: Carregamento dos dados brutos
###  **Transformação**: Limpeza e pré-processamento
###  **Carga**: Exportação para Neo4j

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import os

os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['JAVA_HOME'] = r'C:\Program Files\Eclipse Adoptium\jdk-17.0.15.6-hotspot'

# Configurar Spark Session
spark = SparkSession.builder \
    .appName("FlightDelaysETL") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")
print("Spark Session iniciada com sucesso!")
print(f"Versão do Spark: {spark.version}")
print("Hadoop version:",
      spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

Spark Session iniciada com sucesso!
Versão do Spark: 4.0.0
Hadoop version: 3.4.1


In [2]:
# Definir os caminhos dos arquivos
airlines_path = "data/airlines.csv"
airports_path = "data/airports.csv"
flights_path = "data/flights.parquet"

print("Carregando dados...")

# Carregar os datasets
try:
    # Carregar airlines
    airlines_df = spark.read.csv(airlines_path, header=True, inferSchema=True)
    print("✓ Airlines carregado com sucesso")
    
    # Carregar airports
    airports_df = spark.read.csv(airports_path, header=True, inferSchema=True)
    print("✓ Airports carregado com sucesso")
    
    # Carregar flights
    flights_df = spark.read.parquet(flights_path)
    print("✓ Flights carregado com sucesso")
    
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    raise

Carregando dados...
✓ Airlines carregado com sucesso
✓ Airports carregado com sucesso
✓ Flights carregado com sucesso


#### Análise Inicial dos Dados

In [3]:
# Verificar esquemas dos datasets
print("=== ESQUEMAS DOS DATASETS ===\n")

print("AIRLINES:")
airlines_df.printSchema()
print(f"Registros: {airlines_df.count()}")

print("\nAIRPORTS:")
airports_df.printSchema()
print(f"Registros: {airports_df.count()}")

print("\nFLIGHTS:")
flights_df.printSchema()
print(f"Registros: {flights_df.count()}")

=== ESQUEMAS DOS DATASETS ===

AIRLINES:
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)

Registros: 14

AIRPORTS:
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)

Registros: 322

FLIGHTS:
root
 |-- YEAR: double (nullable = true)
 |-- MONTH: double (nullable = true)
 |-- DAY: double (nullable = true)
 |-- DAY_OF_WEEK: double (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: double (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: double (nullable = true)
 |-- DEPARTURE_TIME: double (nullable = true)
 |-- DEPARTURE_DELAY: double (nullable = true)
 |-- TAXI_OUT: doubl

#### Junção dos Dados

In [4]:
# Fazer join dos dados de voos com airlines e airports
print("Realizando junção dos dados...")

# Join com airlines
flights_with_airlines = flights_df.join(
    airlines_df.select(col("IATA_CODE").alias("AIRLINE_CODE"),
                      col("AIRLINE").alias("AIRLINE_NAME")),
    flights_df.AIRLINE == col("AIRLINE_CODE"),
    "left"
)

# Join com airports de origem
flights_with_origin = flights_with_airlines.join(
    airports_df.select(col("IATA_CODE").alias("ORIGIN_CODE"),
                      col("AIRPORT").alias("ORIGIN_AIRPORT_NAME"),
                      col("CITY").alias("ORIGIN_CITY"),
                      col("STATE").alias("ORIGIN_STATE")),
    flights_with_airlines.ORIGIN_AIRPORT == col("ORIGIN_CODE"),
    "left"
)

# Join com airports de destino
flights_complete = flights_with_origin.join(
    airports_df.select(col("IATA_CODE").alias("DEST_CODE"),
                      col("AIRPORT").alias("DEST_AIRPORT_NAME"),
                      col("CITY").alias("DEST_CITY"),
                      col("STATE").alias("DEST_STATE")),
    flights_with_origin.DESTINATION_AIRPORT == col("DEST_CODE"),
    "left"
)

# CORREÇÃO: Filtrar apenas registros onde o join foi bem-sucedido
# Isso remove registros onde ORIGIN_AIRPORT ou DESTINATION_AIRPORT não são códigos válidos
flights_complete = flights_complete.filter(
    col("ORIGIN_AIRPORT_NAME").isNotNull() &
    col("DEST_AIRPORT_NAME").isNotNull()
)

# Renomear AIRLINE para o nome da companhia aérea e substituir as colunas de aeroportos
flights_complete = (flights_complete
                   .withColumn("AIRLINE", col("AIRLINE_NAME"))
                   .drop("ORIGIN_AIRPORT", "DESTINATION_AIRPORT")  # Remove as colunas originais
                   .withColumnRenamed("ORIGIN_CODE", "ORIGIN_AIRPORT")  # Usa o código validado
                   .withColumnRenamed("DEST_CODE", "DESTINATION_AIRPORT")  # Usa o código validado
                   .drop("AIRLINE_CODE", "AIRLINE_NAME")  # Remove colunas auxiliares
                   )

print(f"Dados unidos com sucesso! Total de registros: {flights_complete.count()}")

# Verificar se ainda existem códigos inválidos
print("Verificando códigos de aeroportos únicos após limpeza:")
print(f"Aeroportos de origem únicos: {flights_complete.select('ORIGIN_AIRPORT').distinct().count()}")
print(f"Aeroportos de destino únicos: {flights_complete.select('DESTINATION_AIRPORT').distinct().count()}")

# Mostrar alguns exemplos dos códigos limpos
print("\nExemplos de aeroportos de origem:")
flights_complete.select("ORIGIN_AIRPORT", "ORIGIN_AIRPORT_NAME").distinct().show(10)

Realizando junção dos dados...
Dados unidos com sucesso! Total de registros: 5332914
Verificando códigos de aeroportos únicos após limpeza:
Aeroportos de origem únicos: 322
Aeroportos de destino únicos: 322

Exemplos de aeroportos de origem:
+--------------+--------------------+
|ORIGIN_AIRPORT| ORIGIN_AIRPORT_NAME|
+--------------+--------------------+
|           ROC|Greater Rochester...|
|           COU|Columbia Regional...|
|           RIC|Richmond Internat...|
|           FAY|Fayetteville Regi...|
|           TUS|Tucson Internatio...|
|           ALO|Waterloo Regional...|
|           ONT|Ontario Internati...|
|           SEA|Seattle-Tacoma In...|
|           MHT|Manchester-Boston...|
|           GPT|Gulfport-Biloxi I...|
+--------------+--------------------+
only showing top 10 rows


#### Verificação de Duplicatas

In [5]:
# Verificar duplicatas
print("=== VERIFICAÇÃO DE DUPLICATAS ===")

total_records = flights_complete.count()

# Contar registros únicos
unique_records = flights_complete.distinct().count()
duplicate_records = total_records - unique_records

print(f"Registros únicos: {unique_records:,}")
print(f"Registros duplicados: {duplicate_records:,}")
print(f"Porcentagem de duplicatas: {(duplicate_records / total_records * 100):.2f}%")

if duplicate_records > 0:
    print("\n⚠️  Removendo duplicatas...")
    flights_complete = flights_complete.distinct()
    print(f"✓ Duplicatas removidas. Registros restantes: {flights_complete.count():,}")

=== VERIFICAÇÃO DE DUPLICATAS ===
Registros únicos: 5,332,914
Registros duplicados: 0
Porcentagem de duplicatas: 0.00%


#### Análise de Valores Nulos

In [6]:
# Análise de valores nulos por coluna
print("=== ANÁLISE DE VALORES NULOS ===")

null_analysis = []
total_rows = flights_complete.count()

for column in flights_complete.columns:
    null_count = flights_complete.filter(col(column).isNull()).count()
    null_percentage = (null_count / total_rows) * 100
    null_analysis.append({
        'column': column,
        'null_count': null_count,
        'null_percentage': null_percentage
    })

# Converter para DataFrame do pandas para melhor visualização
null_df = pd.DataFrame(null_analysis)
null_df = null_df.sort_values('null_percentage', ascending=False)

print(f"Análise de valores nulos (Total de registros: {total_rows:,}):")
print("-" * 60)
for _, row in null_df.iterrows():
    print(f"{row['column']:<25} | {row['null_count']:>10,} | {row['null_percentage']:>8.2f}%")

=== ANÁLISE DE VALORES NULOS ===
Análise de valores nulos (Total de registros: 5,332,914):
------------------------------------------------------------
CANCELLATION_REASON       |  5,245,484 |    98.36%
AIRLINE_DELAY             |  4,329,554 |    81.19%
WEATHER_DELAY             |  4,329,554 |    81.19%
SECURITY_DELAY            |  4,329,554 |    81.19%
AIR_SYSTEM_DELAY          |  4,329,554 |    81.19%
LATE_AIRCRAFT_DELAY       |  4,329,554 |    81.19%
ELAPSED_TIME              |    101,784 |     1.91%
AIR_TIME                  |    101,784 |     1.91%
ARRIVAL_DELAY             |    101,784 |     1.91%
TAXI_IN                   |     89,942 |     1.69%
ARRIVAL_TIME              |     89,942 |     1.69%
WHEELS_ON                 |     89,942 |     1.69%
WHEELS_OFF                |     86,612 |     1.62%
TAXI_OUT                  |     86,612 |     1.62%
DEPARTURE_DELAY           |     83,814 |     1.57%
DEPARTURE_TIME            |     83,814 |     1.57%
TAIL_NUMBER               |     

#### Remoção de Colunas com Mais de 80% de Valores Nulos

In [7]:
# Identificar colunas com mais de 80% de valores nulos
columns_to_remove = null_df[null_df['null_percentage'] > 80]['column'].tolist()

print("=== REMOÇÃO DE COLUNAS COM > 80% DE VALORES NULOS ===")

if columns_to_remove:
    print(f"Colunas a serem removidas ({len(columns_to_remove)}):")
    for col_name in columns_to_remove:
        null_pct = null_df[null_df['column'] == col_name]['null_percentage'].iloc[0]
        print(f"  - {col_name}: {null_pct:.2f}% nulos")
    
    # Remover as colunas
    flights_complete = flights_complete.drop(*columns_to_remove)
    print(f"\n✓ Colunas removidas. Colunas restantes: {len(flights_complete.columns)}")
else:
    print("✓ Nenhuma coluna possui mais de 80% de valores nulos")


=== REMOÇÃO DE COLUNAS COM > 80% DE VALORES NULOS ===
Colunas a serem removidas (6):
  - CANCELLATION_REASON: 98.36% nulos
  - AIRLINE_DELAY: 81.19% nulos
  - WEATHER_DELAY: 81.19% nulos
  - SECURITY_DELAY: 81.19% nulos
  - AIR_SYSTEM_DELAY: 81.19% nulos
  - LATE_AIRCRAFT_DELAY: 81.19% nulos

✓ Colunas removidas. Colunas restantes: 31


#### Seleção das Colunas Uteis para o Banco

In [8]:
print("=== CRIAÇÃO DA COLUNA DATE COM make_date ===")

flights_complete = flights_complete.withColumn(
    "DATE",
    make_date(
        col("YEAR").cast("int"),
        col("MONTH").cast("int"),
        col("DAY").cast("int")
    )
)

print("✓ Coluna DATE criada com sucesso (ou NULL se inválida)")
flights_complete.select("YEAR", "MONTH", "DAY", "DATE").show(5)


=== CRIAÇÃO DA COLUNA DATE COM make_date ===
✓ Coluna DATE criada com sucesso (ou NULL se inválida)
+------+-----+---+----------+
|  YEAR|MONTH|DAY|      DATE|
+------+-----+---+----------+
|2015.0|  1.0|1.0|2015-01-01|
|2015.0|  1.0|1.0|2015-01-01|
|2015.0|  1.0|1.0|2015-01-01|
|2015.0|  1.0|1.0|2015-01-01|
|2015.0|  1.0|1.0|2015-01-01|
+------+-----+---+----------+
only showing top 5 rows


In [9]:
# Selecionar apenas as colunas especificadas
selected_columns = [
    'DATE','AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
    'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
    'SCHEDULED_TIME', 'ELAPSED_TIME'
]

print("=== SELEÇÃO DE COLUNAS ESPECÍFICAS ===")
print(f"Colunas selecionadas ({len(selected_columns)}):")
for col_select in selected_columns:
    print(f"  - {col_select}")

# Verificar se todas as colunas existem
existing_columns = flights_complete.columns
missing_columns = [col for col in selected_columns if col not in existing_columns]

if missing_columns:
    print(f"\n⚠️  Colunas não encontradas: {missing_columns}")
    # Remover colunas não encontradas da lista
    selected_columns = [col for col in selected_columns if col in existing_columns]
    print(f"Usando apenas colunas existentes: {selected_columns}")

# Selecionar apenas as colunas desejadas
flights_selected = flights_complete.select(*selected_columns)

print(f"\n✓ Dataset final com {len(selected_columns)} colunas e {flights_selected.count():,} registros")

=== SELEÇÃO DE COLUNAS ESPECÍFICAS ===
Colunas selecionadas (12):
  - DATE
  - AIRLINE
  - ORIGIN_AIRPORT
  - DESTINATION_AIRPORT
  - SCHEDULED_DEPARTURE
  - DEPARTURE_TIME
  - DEPARTURE_DELAY
  - SCHEDULED_ARRIVAL
  - ARRIVAL_TIME
  - ARRIVAL_DELAY
  - SCHEDULED_TIME
  - ELAPSED_TIME

✓ Dataset final com 12 colunas e 5,332,914 registros


#### Análise Final de Valores Nulos

In [10]:
# Análise final de valores nulos nas colunas selecionadas
print("=== ANÁLISE FINAL DE VALORES NULOS ===")

final_null_analysis = []
total_rows_final = flights_selected.count()

# Importar explicitamente para evitar conflitos
from pyspark.sql.functions import col as spark_col

for col_name in flights_selected.columns:
    null_count = flights_selected.filter(spark_col(col_name).isNull()).count()
    null_percentage = (null_count / total_rows_final) * 100
    final_null_analysis.append({
        'column': col_name,
        'null_count': null_count,
        'null_percentage': null_percentage
    })

# Converter para DataFrame do pandas
final_null_df = pd.DataFrame(final_null_analysis)
final_null_df = final_null_df.sort_values('null_percentage', ascending=False)

print(f"Análise de valores nulos no dataset final (Total: {total_rows_final:,}):")
print("-" * 60)
for _, row in final_null_df.iterrows():
    print(f"{row['column']:<25} | {row['null_count']:>10,} | {row['null_percentage']:>8.2f}%")

# Calcular total de linhas com pelo menos um valor nulo
# Usar uma abordagem mais robusta
null_condition = None
for col_name in flights_selected.columns:
    if null_condition is None:
        null_condition = spark_col(col_name).isNull()
    else:
        null_condition = null_condition | spark_col(col_name).isNull()

rows_with_nulls = flights_selected.filter(null_condition).count()

null_rows_percentage = (rows_with_nulls / total_rows_final) * 100
print(f"\nLinhas com pelo menos um valor nulo: {rows_with_nulls:,} ({null_rows_percentage:.2f}%)")

=== ANÁLISE FINAL DE VALORES NULOS ===
Análise de valores nulos no dataset final (Total: 5,332,914):
------------------------------------------------------------
ARRIVAL_DELAY             |    101,784 |     1.91%
ELAPSED_TIME              |    101,784 |     1.91%
ARRIVAL_TIME              |     89,942 |     1.69%
DEPARTURE_TIME            |     83,814 |     1.57%
DEPARTURE_DELAY           |     83,814 |     1.57%
SCHEDULED_TIME            |          6 |     0.00%
AIRLINE                   |          0 |     0.00%
DATE                      |          0 |     0.00%
DESTINATION_AIRPORT       |          0 |     0.00%
ORIGIN_AIRPORT            |          0 |     0.00%
SCHEDULED_ARRIVAL         |          0 |     0.00%
SCHEDULED_DEPARTURE       |          0 |     0.00%

Linhas com pelo menos um valor nulo: 101,784 (1.91%)


#### Remoção de Linhas com Valores Nulos

In [11]:
print(f"Removendo {rows_with_nulls:,} linhas com valores nulos...")
    
# Criar dataset limpo (sem valores nulos)
flights_clean = flights_selected.na.drop()

final_count = flights_clean.count()
removed_rows = total_rows_final - final_count

print(f"✓ Remoção concluída:")
print(f"  - Registros removidos: {removed_rows:,}")
print(f"  - Registros restantes: {final_count:,}")
print(f"  - Porcentagem removida: {(removed_rows / total_rows_final * 100):.2f}%")
print(f"  - Porcentagem mantida: {(final_count / total_rows_final * 100):.2f}%")

Removendo 101,784 linhas com valores nulos...
✓ Remoção concluída:
  - Registros removidos: 101,784
  - Registros restantes: 5,231,130
  - Porcentagem removida: 1.91%
  - Porcentagem mantida: 98.09%


#### Transformações e Limpeza de Dados

In [12]:
print("=== TRANSFORMAÇÕES E LIMPEZA DE DADOS ===")

# Verificar tipos de dados atuais
print("Tipos de dados atuais:")
flights_clean.printSchema()

# Aplicar transformações de limpeza
print("\nAplicando transformações...")

flights_transformed = flights_clean.withColumn(
    "DATE",
    date_format(col("DATE"), "yyyy-MM-dd")
)

# Converter colunas de tempo para tipos apropriados
flights_transformed = flights_clean.withColumn("SCHEDULED_DEPARTURE", col("SCHEDULED_DEPARTURE").cast("integer")) \
                                  .withColumn("DEPARTURE_TIME", col("DEPARTURE_TIME").cast("integer")) \
                                  .withColumn("DEPARTURE_DELAY", col("DEPARTURE_DELAY").cast("integer")) \
                                  .withColumn("SCHEDULED_ARRIVAL", col("SCHEDULED_ARRIVAL").cast("integer")) \
                                  .withColumn("ARRIVAL_TIME", col("ARRIVAL_TIME").cast("integer")) \
                                  .withColumn("ARRIVAL_DELAY", col("ARRIVAL_DELAY").cast("integer")) \
                                  .withColumn("SCHEDULED_TIME", col("SCHEDULED_TIME").cast("integer")) \
                                  .withColumn("ELAPSED_TIME", col("ELAPSED_TIME").cast("integer"))


# Limpar e padronizar códigos de aeroportos
flights_transformed = flights_transformed.withColumn("ORIGIN_AIRPORT", upper(trim(col("ORIGIN_AIRPORT")))) \
                                        .withColumn("DESTINATION_AIRPORT", upper(trim(col("DESTINATION_AIRPORT")))) \
                                        .withColumn("AIRLINE", trim(col("AIRLINE")))

# Filtrar registros inconsistentes (delays muito extremos)
flights_transformed = flights_transformed.filter(
    (col("DEPARTURE_DELAY") >= -60) & (col("DEPARTURE_DELAY") <= 1440) &  # -60 min a 24h
    (col("ARRIVAL_DELAY") >= -60) & (col("ARRIVAL_DELAY") <= 1440)
)

print(f"✓ Transformações aplicadas. Registros finais: {flights_transformed.count():,}")



=== TRANSFORMAÇÕES E LIMPEZA DE DADOS ===
Tipos de dados atuais:
root
 |-- DATE: date (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: double (nullable = true)
 |-- DEPARTURE_TIME: double (nullable = true)
 |-- DEPARTURE_DELAY: double (nullable = true)
 |-- SCHEDULED_ARRIVAL: double (nullable = true)
 |-- ARRIVAL_TIME: double (nullable = true)
 |-- ARRIVAL_DELAY: double (nullable = true)
 |-- SCHEDULED_TIME: double (nullable = true)
 |-- ELAPSED_TIME: double (nullable = true)


Aplicando transformações...
✓ Transformações aplicadas. Registros finais: 5,230,735


#### Conexão e Carga para Neo4j

In [13]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from airport_flight_system import AirportFlightSystem

In [15]:
# Configurações do Neo4j
NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"  # Ajuste conforme sua configuração

# Inicializar o sistema
try:
    flight_system = AirportFlightSystem(
        uri=NEO4J_URI,
        user=NEO4J_USER,
        password=NEO4J_PASSWORD
    )
    print("✅ Sistema inicializado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao inicializar sistema: {e}")
    raise

INFO:airport_flight_system:✅ Conectado ao Neo4j com sucesso! Database: flights-db
INFO:airport_flight_system:🔍 Teste de conexão no database 'flights-db': 1


✅ Sistema inicializado com sucesso!


### Configuração do Banco de Dados

In [16]:
print("🔧 Configurando banco de dados...")
flight_system.setup_database()
print("✅ Configuração concluída!\n")

print("📊 Status inicial do banco de dados:")
initial_info = flight_system.get_database_info()
print(f"Nós: {initial_info['total_nodes']}")
print(f"Relacionamentos: {initial_info['total_relationships']}")
print(f"Labels: {initial_info['labels']}")
print(f"Tipos de relacionamento: {initial_info['relationship_types']}\n")


INFO:airport_flight_system:🔧 Configurando banco de dados 'flights-db'...
INFO:airport_flight_system:ℹ Constraint airport_code_unique já existe: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=3, name='airport_code_unique', type='NODE PROPERTY UNIQUENESS', schema=(:Airport {code}), ownedIndex=2 )'.}
INFO:airport_flight_system:ℹ Índice airport_code_index já existe: {code: Neo.ClientError.Schema.ConstraintAlreadyExists} {message: There is a uniqueness constraint on (:Airport {code}), so an index is already created that matches this.}
INFO:airport_flight_system:ℹ Índice flight_date_index já existe: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=4, name='flight_date_index', type='RANGE', schema=()-[:FLIGHT {date}]-(), indexProvider='range-1.0' )'.}
INFO:airport_flight_system:ℹ Índice flight_airline_index já existe: {code: Neo.ClientEr

🔧 Configurando banco de dados...
✅ Configuração concluída!

📊 Status inicial do banco de dados:


INFO:airport_flight_system:📊 Info do banco 'flights-db': 0 nós, 0 relacionamentos


Nós: 0
Relacionamentos: 0
Labels: ['Airport']
Tipos de relacionamento: ['FLIGHT']



### apagando pro db começar sem niguem garantidamente

In [17]:
#apagando todos os dados do banco
flight_system.clear_database()

INFO:airport_flight_system:🧹 Banco de dados 'flights-db' limpo


### Criar Aeroportos Únicos

In [18]:
print("🛫 Criando aeroportos únicos...")

# Obter aeroportos únicos do DataFrame
origin_airports = flights_transformed.select("ORIGIN_AIRPORT").distinct().collect()
destination_airports = flights_transformed.select("DESTINATION_AIRPORT").distinct().collect()

# Combinar e remover duplicatas
all_airports = set()
for row in origin_airports:
    if row.ORIGIN_AIRPORT:
        all_airports.add(row.ORIGIN_AIRPORT)
for row in destination_airports:
    if row.DESTINATION_AIRPORT:
        all_airports.add(row.DESTINATION_AIRPORT)

print(f"Total de aeroportos únicos encontrados: {len(all_airports)}")

# Criar aeroportos no Neo4j (com dados básicos)
airports_created = 0
for airport_code in all_airports:
    try:
        flight_system.create_airport(
            code=airport_code,
            name=f"Airport {airport_code}",  # Nome genérico
            city=f"City {airport_code}",     # Cidade genérica
            state=f"State {airport_code}",   # Estado genérico
            country="USA"
        )
        airports_created += 1
    except Exception as e:
        print(f"Erro ao criar aeroporto {airport_code}: {e}")

print(f"✅ {airports_created} aeroportos criados com sucesso!\n")

🛫 Criando aeroportos únicos...
Total de aeroportos únicos encontrados: 322


INFO:airport_flight_system:✅ Aeroporto TRI criado/atualizado: Airport TRI
INFO:airport_flight_system:✅ Aeroporto FAR criado/atualizado: Airport FAR
INFO:airport_flight_system:✅ Aeroporto MSN criado/atualizado: Airport MSN
INFO:airport_flight_system:✅ Aeroporto BIS criado/atualizado: Airport BIS
INFO:airport_flight_system:✅ Aeroporto PNS criado/atualizado: Airport PNS
INFO:airport_flight_system:✅ Aeroporto ACK criado/atualizado: Airport ACK
INFO:airport_flight_system:✅ Aeroporto MAF criado/atualizado: Airport MAF
INFO:airport_flight_system:✅ Aeroporto BQN criado/atualizado: Airport BQN
INFO:airport_flight_system:✅ Aeroporto EVV criado/atualizado: Airport EVV
INFO:airport_flight_system:✅ Aeroporto ORD criado/atualizado: Airport ORD
INFO:airport_flight_system:✅ Aeroporto FLG criado/atualizado: Airport FLG
INFO:airport_flight_system:✅ Aeroporto DAY criado/atualizado: Airport DAY
INFO:airport_flight_system:✅ Aeroporto MHT criado/atualizado: Airport MHT
INFO:airport_flight_system:✅ Aeroporto

✅ 322 aeroportos criados com sucesso!



### Inserir Voos (Processamento em Lotes)

In [23]:
print("✈️ Inserindo voos no Neo4j...")

# Obter contagem total para acompanhar progresso
total_flights = flights_transformed.count()
print(f"Total de voos para inserir: {total_flights}")

# Para datasets grandes, usar particionamento
batch_size = 5000
flights_created = 0
errors = 0

print(f"Processando voos em lotes de {batch_size}...")

# Adicionar uma coluna de ID sequencial para facilitar o processamento por lotes
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Adicionar row_number para processar em lotes
windowSpec = Window.orderBy(flights_transformed.columns[0])  # Usar primeira coluna para ordenação
flights_with_id = flights_transformed.withColumn("row_id", row_number().over(windowSpec))

# Calcular número total de lotes
total_batches = (total_flights + batch_size - 1) // batch_size
print(f"Total de lotes a processar: {total_batches}")

# Processar em lotes usando filtro por row_id
for batch_num in range(total_batches):
    start_row = batch_num * batch_size + 1
    end_row = (batch_num + 1) * batch_size
    if end_row > total_flights:
        end_row = total_flights

    print(f"Processando lote {batch_num + 1}/{total_batches} (linhas {start_row} a {end_row})...")

    try:
        # Filtrar o lote atual usando row_id
        batch_df = flights_with_id.filter(
            (flights_with_id.row_id >= start_row) &
            (flights_with_id.row_id <= end_row)
        ).drop("row_id")  # Remover a coluna auxiliar

        # Verificar se há dados no lote
        batch_count = batch_df.count()
        if batch_count == 0:
            print(f"Lote {batch_num + 1} vazio, pulando...")
            continue

        print(f"Lote {batch_num + 1} coletado com {batch_count} registros. Inserindo no Neo4j...")

        # Converter para pandas para processamento mais eficiente
        try:
            import pandas as pd
            batch_pandas = batch_df.toPandas()

            # Processar cada registro do lote
            for idx, row in batch_pandas.iterrows():
                try:
                    # Verificar se os dados obrigatórios existem
                    if pd.isna(row.get('ORIGIN_AIRPORT')) or pd.isna(row.get('DESTINATION_AIRPORT')):
                        errors += 1
                        continue

                    flight_data = {
                        'date': str(row.get('DATE', '2015-01-01')),
                        'airline': row.get('AIRLINE', 'Unknown'),
                        'origin_airport': row.get('ORIGIN_AIRPORT'),
                        'destination_airport': row.get('DESTINATION_AIRPORT'),
                        'scheduled_departure': float(row.get('SCHEDULED_DEPARTURE', 0.0)) if pd.notna(row.get('SCHEDULED_DEPARTURE')) else 0.0,
                        'departure_time': float(row.get('DEPARTURE_TIME', 0.0)) if pd.notna(row.get('DEPARTURE_TIME')) else 0.0,
                        'departure_delay': float(row.get('DEPARTURE_DELAY', 0.0)) if pd.notna(row.get('DEPARTURE_DELAY')) else 0.0,
                        'scheduled_arrival': float(row.get('SCHEDULED_ARRIVAL', 0.0)) if pd.notna(row.get('SCHEDULED_ARRIVAL')) else 0.0,
                        'arrival_time': float(row.get('ARRIVAL_TIME', 0.0)) if pd.notna(row.get('ARRIVAL_TIME')) else 0.0,
                        'arrival_delay': float(row.get('ARRIVAL_DELAY', 0.0)) if pd.notna(row.get('ARRIVAL_DELAY')) else 0.0,
                        'scheduled_time': float(row.get('SCHEDULED_TIME', 0.0)) if pd.notna(row.get('SCHEDULED_TIME')) else 0.0,
                        'elapsed_time': float(row.get('ELAPSED_TIME', 0.0)) if pd.notna(row.get('ELAPSED_TIME')) else 0.0
                    }

                    flight_system.create_flight(flight_data)
                    flights_created += 1

                except Exception as e:
                    errors += 1
                    if errors <= 20:  # Mostrar mais erros para debug
                        print(f"Erro no voo {flights_created + errors}: {e}")

        except ImportError:
            # Fallback para collect() se pandas não estiver disponível
            print("Pandas não disponível, usando collect()...")
            batch_data = batch_df.collect()

            for row in batch_data:
                try:
                    # Verificar se os dados obrigatórios existem
                    if not row.ORIGIN_AIRPORT or not row.DESTINATION_AIRPORT:
                        errors += 1
                        continue

                    flight_data = {
                        'date': str(row.DATE) if row.DATE else '2015-01-01',
                        'airline': row.AIRLINE if row.AIRLINE else 'Unknown',
                        'origin_airport': row.ORIGIN_AIRPORT,
                        'destination_airport': row.DESTINATION_AIRPORT,
                        'scheduled_departure': float(row.SCHEDULED_DEPARTURE) if row.SCHEDULED_DEPARTURE is not None else 0.0,
                        'departure_time': float(row.DEPARTURE_TIME) if row.DEPARTURE_TIME is not None else 0.0,
                        'departure_delay': float(row.DEPARTURE_DELAY) if row.DEPARTURE_DELAY is not None else 0.0,
                        'scheduled_arrival': float(row.SCHEDULED_ARRIVAL) if row.SCHEDULED_ARRIVAL is not None else 0.0,
                        'arrival_time': float(row.ARRIVAL_TIME) if row.ARRIVAL_TIME is not None else 0.0,
                        'arrival_delay': float(row.ARRIVAL_DELAY) if row.ARRIVAL_DELAY is not None else 0.0,
                        'scheduled_time': float(row.SCHEDULED_TIME) if row.SCHEDULED_TIME is not None else 0.0,
                        'elapsed_time': float(row.ELAPSED_TIME) if row.ELAPSED_TIME is not None else 0.0
                    }

                    flight_system.create_flight(flight_data)
                    flights_created += 1

                except Exception as e:
                    errors += 1
                    if errors <= 20:
                        print(f"Erro no voo {flights_created + errors}: {e}")

        # Log do progresso
        processed_count = flights_created + errors
        progress_percent = (processed_count / total_flights) * 100
        print(f"Progresso: {processed_count}/{total_flights} voos processados ({progress_percent:.1f}%)")
        print(f"✅ Lote {batch_num + 1} concluído! {flights_created} voos criados até agora, {errors} erros")

        # Pausa para não sobrecarregar o Neo4j
        import time
        time.sleep(0.2)

    except Exception as e:
        print(f"Erro no lote {batch_num + 1}: {e}")
        print("Tentando continuar com o próximo lote...")
        continue

print(f"\n✅ Inserção concluída! {flights_created} voos criados, {errors} erros")
if flights_created + errors > 0:
    print(f"Taxa de sucesso: {(flights_created / (flights_created + errors)) * 100:.1f}%")
else:
    print("Nenhum voo foi processado.")

# Verificar se todos os registros foram processados
if flights_created + errors < total_flights:
    print(f"⚠️  ATENÇÃO: Apenas {flights_created + errors} de {total_flights} registros foram processados!")
    print("Isso pode indicar um problema no processamento por lotes.")
else:
    print("✅ Todos os registros foram processados com sucesso!")

✈️ Inserindo voos no Neo4j...
Total de voos para inserir: 5230735
Processando voos em lotes de 5000...
Total de lotes a processar: 1047
Processando lote 1/1047 (linhas 1 a 5000)...
Lote 1 coletado com 5000 registros. Inserindo no Neo4j...
Progresso: 5000/5230735 voos processados (0.1%)
✅ Lote 1 concluído! 5000 voos criados até agora, 0 erros
Processando lote 2/1047 (linhas 5001 a 10000)...
Lote 2 coletado com 5000 registros. Inserindo no Neo4j...
Progresso: 10000/5230735 voos processados (0.2%)
✅ Lote 2 concluído! 10000 voos criados até agora, 0 erros
Processando lote 3/1047 (linhas 10001 a 15000)...
Lote 3 coletado com 5000 registros. Inserindo no Neo4j...
Progresso: 15000/5230735 voos processados (0.3%)
✅ Lote 3 concluído! 15000 voos criados até agora, 0 erros
Processando lote 4/1047 (linhas 15001 a 20000)...
Lote 4 coletado com 5000 registros. Inserindo no Neo4j...
Progresso: 20000/5230735 voos processados (0.4%)
✅ Lote 4 concluído! 20000 voos criados até agora, 0 erros
Processando 

In [25]:
print("\n🔌 Fechando conexão com Neo4j...")
flight_system.close()
print("✅ Conexão fechada com sucesso!")

INFO:airport_flight_system:🔌 Conexão com 'flights-db' fechada



🔌 Fechando conexão com Neo4j...
✅ Conexão fechada com sucesso!
